In [ ]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [ ]:
class Receiver:
    def __init__(self, parameters):
        self.private_key = None
        self.public_key = None
        self.shared_key = None
        self.parameters = parameters
        self.get_private_key(parameters)
        self.get_public_key()

    def get_private_key(self, parameters):
        self.private_key = parameters.generate_private_key()
    
    def get_public_key(self):
        self.public_key = self.private_key.public_key()

    def get_derived_key(self, emitter_public_key):
        key_int = self.private_key.exchange(emitter_public_key)
        # Derivar chave
        self.shared_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=None,
        ).derive(key_int)


    def verify_Auth(self, message, signature):
        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(message)
        try: 
            h.verify(signature)
            return True
        except cryptography.exceptions.InvalidSignature:
            return False

    def decode (self, ciphertext, nonce, aad):
        aesgcm = AESGCM(self.shared_key)
        try: 
            plaintext = aesgcm.decrypt(nonce, ciphertext, aad)
        except cryptography.exceptions.InvalidTag:
            return None
        return plaintext.decode('utf-8')
    

    def divide_data(self, data):
        # sign + nonce + ad + ciphertext
        # 0 a 31 - signature : 32 bytes
        # 32 a 47 - nonce : 16 bytes
        # 48 a 59 - ad para derivar
        # a partir do 60 - texto cifrado
        sign = data[:32]
        nonce = data[32:48]
        ad = data[48:60]
        texto_cifrado = data[60:] 
        return sign, nonce, ad, texto_cifrado


    #função que lê a mensagem 
    def readMessage(self, data, menSig):
        sign, nonce, ad, texto_cifrado = self.divide_data(data)
        #verifica se o autor da mensagem é o esperado
        if(self.verify_Auth(menSig, sign)):
            aad = ad + nonce
            #decifra o texto 
            plaintext = self.decode(texto_cifrado, nonce, aad)
            print("Texto decifrado: ")
            print(plaintext)
        else:
            raise Exception("Autenticidade tem falhas!")